<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/C4_FAQs_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -Uq transformers sentence-transformers datasets sentencepiece

In [1]:
from datasets import load_dataset
from tqdm.auto import tqdm
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize # We need a fast sentence-tokenizer
from typing import List
import pandas as pd
import random
from huggingface_hub import notebook_login
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
"""
Structure Basic (list of dict below):
{
  "faq_pairs": [
    {"question": question, "answer": answer},
    {"question": question, "answer": answer}
  ]
}
"""

In [2]:
dataset = load_dataset("c4", "en", split="train", streaming=True)

In [20]:
def tokenize_text(text: str):
  sentence_tokens = sent_tokenize(text)
  return sentence_tokens

def num_sentence_token(text: str):
  return len(tokenize_text(text))

def num_word_tokens(text: str):
  return len(word_tokenize(text))

def get_title(text: str):
  sentence_tokens = sent_tokenize(text)
  title = " ".join(sentence_tokens[:min(3, len(sentence_tokens))])
  return title

def flow(sentence_tokens: List[str]):
  initial_question_found = False
  questions, answers = [], []
  for i, sent in enumerate(sentence_tokens):
    if sent.endswith("?"):
      questions.append(sent)
      answer_j = []
      for sent_j in sentence_tokens[i+1:]:
        if not sent_j.endswith("?"):
          answer_j.append(sent_j)
        else:
          break
      answer_j_text = " ".join(answer_j)
      answers.append(answer_j_text)
  
  qa_pairs = list(zip(questions, answers))
  qa_pairs = list(filter(lambda x: num_word_tokens(x[0]) > 2 and \
                         num_word_tokens(x[1]) > 16, qa_pairs))
  return qa_pairs

In [21]:
def save_dataset_list_and_flush(dataset_list, count_save):
  df = pd.DataFrame.from_records(dataset_list)
  df.to_json(f"c4-faqs_{count_save}.jsonl", orient="records", lines=True)

In [23]:
count_faqs = 0
done_urls = set()
limit = 1000
checkpoint_limit = 10
progress_bar_total = tqdm(range(364868892), desc="total")
progress_bar_faqs = tqdm(range(limit), desc="faq-pages")
progress_bar_pairs = tqdm(range(10**6), desc="qa-pairs")
dataset_list = []
all_questions, all_answers = [], []
count_save = 1
for dict_ in dataset:
  progress_bar_total.update(1)
  if count_faqs == limit:
    break

  url = dict_["url"]
  text = dict_["text"]
  title = get_title(text)
  if "faq" in url.lower() or "faq" in title.lower():
    dict_ = {}
    # dict_["url"] = url
    # dict_["text"] = text
    done_urls.add(url)
    qa_pairs = flow(tokenize_text(text))
    if qa_pairs:
      count_faqs += 1
      progress_bar_faqs.update(1)
      questions, answers = zip(*qa_pairs)
      questions, answers = list(questions), list(answers)
      faq_list = [{"question": q, "answer": a} for q, a in zip(questions, answers)]
      dict_["faq_pairs"] = faq_list
      progress_bar_pairs.update(len(faq_list))
      dataset_list.append(dict_)

      if dataset_list and not (len(dataset_list) % checkpoint_limit):
        save_dataset_list_and_flush(dataset_list, count_save)
        dataset_list.clear()
        count_save += 1
dataset_list.clear()

total:   0%|          | 0/364868892 [00:00<?, ?it/s]

faq-pages:   0%|          | 0/1000 [00:00<?, ?it/s]

qa-pairs:   0%|          | 0/1000000 [00:00<?, ?it/s]

In [24]:
# ! rm *.jsonl
hf_dataset = load_dataset("json", data_files="*.jsonl", split="train")

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-426aaad67ffc049e/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


In [25]:
print(f"Total filtered faq pages: {hf_dataset.num_rows}")

Total filtered faq pages: 1000


In [26]:
# Visualize a random dataset_list entry
random_index = random.sample(range(hf_dataset.num_rows), 1)[0]
hf_dataset[random_index]

{'faq_pairs': [{'question': 'Q: My Controller shows the zones working, but here is no water coming out of the sprinklers?',
   'answer': 'A: Generally this happens after a rainfall when the rain sensor is full of water, the main shut off valve is closed or one or more of the ball valves on tech RPZ is closed. Please see our how to section on turning your main supply On / Off.'},
  {'question': 'Q: Where is my main shut-off valve for the system?',
   'answer': 'A: It is usually located near the water meter or by the hot water tank, marked with an orange Western Irrigation tag. A: Refer to your owner’s manual for specific programming instructions, Pro-C manual page 18, ICC manual page 27, Icore manual page 16. A: First make sure there is power at the outlet. If there is power at the outlet then the controller might need to be replaced. Call Western Irrigation for service. A: When a program finishes watering and starts again, we call it repeat cycling. A common reason for a controller to 

In [19]:
total_faq_pairs = sum([len(x["faq_pairs"]) for x in hf_dataset])
print(f"Total faq pairs: {total_faq_pairs}")

Total faq pairs: 679


In [ ]:
hf_dataset

In [ ]:
notebook_login()

In [ ]:
hf_dataset.push_to_hub("c4-faqs", private=True)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]